# Introduktion til sentiment analysis

## Hvad er sentiment analysis?

> "[...] the field of study that analyzes people’s opinions, sentiments, appraisals, attitudes, and emotions toward entities and their attributes expressed in written text" - Liu 2015

**Formål:** Identificere positive og/eller negative holdninger/følelser i tekst samt hvad, disse holdninger/følelser er rettet imod.

**Eksempler på brug:**

- Undersøge sociale bevægelser på internettet
- Markedsprædiktion
- Policy-evaluering

**Relateret:** opinion mining, opinion analysis, opinion extraction, sentiment mining, subjectivity analysis, affect analysis, emotion analysis, review mining

## Sentiment analysis - Tre overordnede teknikker

**A: Ordbogs- og regelbaseret sentiment analysis ("lexicon and rule-based)**

- Model der grundlæggende bygger på et opslagsværk over, hvorvidt et ord er negativt eller ej (fx "hader" -> negativt, "elsker" -> positivt)
- Kan evt. udbygges med regler sådan, at der tages højde for ord i kontekst (fx "hader ikke" -> neutralt)
- Fordele: inspicérbar, alsidighed
- Ulemper: tager i meget begrænset omfang højde for kontekst

**B: Præ-trænede sentiment classifiers**

- Model der er trænet på eksisterende tekststykker, som er tagget efter holdningen/følelsen udtrykt i tekststykket (positivt, negativt, neutralt)
- Gør brug af machine learning i en eller anden omfang (random forests, naive bayes, neural networks)
- Fordele: tager i højere grad højde for kontekst, (ofte) stort træningsdatasæt
- Ulemper: vanskelig at inspicere

**C: Selv-trænede sentiment classifiers**

- Model der trænes på taggede tekststykker i det materiale, som skal undersøges
- Gør brug af analytikerens egen kendskab til området og materialet til at udlede sentiment
- Fordele: høj kontekstnøjagtighed, til dels inspicerbar
- Ulemper: (sandsynligvis) lille træningsdatasæt, kræver manuel tagging

**Generelle svagheder**

- Kontekst: Modeller er for det meste trænet på data, der deler en eller anden kontekst (tid, genre, sprog)
- Ironi/sarkasme: Vanskeligt at træne modeller til at genkende dette ud fra tekst alene

## Sentiment analysis i Python 1: DaCy

En implementation af sentiment analysis i Python er gennem DaCy: https://centre-for-humanities-computing.github.io/DaCy/

DaCy er en komplet dansk sprogmodel-løsning baseret på spaCy. DaCy indeholder en wrapper til `senda`, som er en sentiment analyzer baseret på en *transformer-model (neural netværk)*: https://github.com/cjhutto/vaderSentiment#ports-to-other-programming-languages.

`senda`-modellen er udviklet af EkstraBladet og er trænet på twitter-data: https://github.com/ebanalyse/senda

### Download og indlæsning

DaCy fungerer mere eller mindre præcis ligesom spaCy. For at bruge den, gør vi følgende:

1. Indlæser DaCy
2. Downloader sprogmodel
3. Indlæser sprogmodel
4. Tilføjer senda til modellens pipeline

In [39]:
import dacy
from dacy.sentiment import add_senda

#dacy.download_model(model="da_dacy_medium_tft-0.0.0")

In [38]:
nlp = dacy.load("da_dacy_medium_tft-0.0.0")
add_senda(nlp, force_extension = True)

### Brug af sentiment analyzer

Sentiment analyzeren bruges på et "doc" objekt (et spaCy dokument). Sådan et dannes ligsesom i spaCy. Vi gør følgende:

- Danne spaCy dokument (med sprogmodel fra DaCy)
- Undersøge sentiment (under attribut `._.polarity` og `._.polarity_prop`

In [45]:
doc = nlp("Den her kaffe er det værste gang tynde pis")

print(doc._.polarity, 
      "\n",
      doc._.polarity_prop)

negative 
 {'prop': array([0.824, 0.121, 0.055], dtype=float32), 'labels': ['negative', 'neutral', 'positive']}


### Sentiment analyzer på dataframe

Ved brug af en kort wrapper-funktion og `.apply` kan sentiment analysis udføres på tekst i en dataframe. Vi gør følgende:

1. Indlæser twitter-data (Folketingsmedlemmers tweets)
2. Danner wrapper-funktion
3. Bruger funktion med apply

In [46]:
import pandas as pd

tweets_df = pd.read_csv('https://raw.githubusercontent.com/CALDISS-AAU/course_ddf/master/datasets/poltweets_sample.csv')
tweets_df.head()

,created_at,id,full_text,is_quote_status,retweet_count,favorite_count,favorited,retweeted,is_retweet,hashtags,urls,user_followers_count,party
0,2020-10-21 14:48:39+00:00,1318927184111730700,Er på vej i miljøministeriet for at foreslå at...,False,13,47,False,False,False,['dkgreen'],[],4064,Alternativet
1,2019-06-02 20:03:20+00:00,1135275725592891400,@nielscallesoe @helenehagel @alternativet_ Det...,False,0,1,False,False,False,[],[],4064,Alternativet
2,2016-03-10 09:07:52+00:00,707855478320189400,"Vi står sammen, smiler Løkke på KL-topmøde og ...",False,13,14,False,False,False,"['dkpol', 'KLtop16']",[],4064,Alternativet
3,2019-04-07 19:59:03+00:00,1114980930467315700,@AnnaBylov @EU_Spring @rasmusnordqvist 💚,False,0,2,False,False,False,[],[],4064,Alternativet
4,2017-05-28 09:59:26+00:00,868768670427828200,Der er ikke noget alternativ til at Alternativ...,False,6,28,False,False,False,['LMÅ17'],"[{'url': 'https://t.co/3MCdZZGKRq', 'expanded_...",4064,Alternativet


In [47]:
def simple_sentiment(text):
    doc = nlp(text)
    polarity = doc._.polarity
    
    return(polarity)

In [49]:
tweets_alternativet = tweets_df.loc[tweets_df['party'] == 'Alternativet', :]
tweets_alternativet['polarity'] = tweets_alternativet['full_text'].apply(simple_sentiment)

tweets_alternativet.head()

C:\Users\kgk\.conda\envs\dacy\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,created_at,id,full_text,is_quote_status,retweet_count,favorite_count,favorited,retweeted,is_retweet,hashtags,urls,user_followers_count,party,polarity
0,2020-10-21 14:48:39+00:00,1318927184111730700,Er på vej i miljøministeriet for at foreslå at...,False,13,47,False,False,False,['dkgreen'],[],4064,Alternativet,neutral
1,2019-06-02 20:03:20+00:00,1135275725592891400,@nielscallesoe @helenehagel @alternativet_ Det...,False,0,1,False,False,False,[],[],4064,Alternativet,negative
2,2016-03-10 09:07:52+00:00,707855478320189400,"Vi står sammen, smiler Løkke på KL-topmøde og ...",False,13,14,False,False,False,"['dkpol', 'KLtop16']",[],4064,Alternativet,neutral
3,2019-04-07 19:59:03+00:00,1114980930467315700,@AnnaBylov @EU_Spring @rasmusnordqvist 💚,False,0,2,False,False,False,[],[],4064,Alternativet,neutral
4,2017-05-28 09:59:26+00:00,868768670427828200,Der er ikke noget alternativ til at Alternativ...,False,6,28,False,False,False,['LMÅ17'],"[{'url': 'https://t.co/3MCdZZGKRq', 'expanded_...",4064,Alternativet,negative


In [52]:
tweets_alternativet['polarity'].value_counts()

negative    191
neutral     173
positive    136
Name: polarity, dtype: int64

## Sentiment analysis  i Python 2: TextBlob

En implementation af sentiment analysis i Python er med TextBlob: https://textblob.readthedocs.io/en/

TextBlob er en sentiment analyzer *model baseret på naive bayes (machine learnig)*,

**Bemærk:** TextBlob egner sig primært til engelsk.

### Indlæsning og brug

TextBlob fungerer på mange måder ligesom spaCy. Man omdanner tekststykket til et "blob" objekt, der indeholder forskellige attributter - herunder sentiment. Vi gør følgende

1. Indlæser TextBlob
2. Danne "blob" objekt ud fra tekststykke
3. Undersøge sentiment (under attribut `sentiment`)

In [20]:
from textblob import TextBlob

blob = TextBlob("This coffee is just absolutely horrible. I hate it.")
print(blob.sentiment)

Sentiment(polarity=-0.9, subjectivity=0.95)


Sentiment-scoren ses af "polarity = -0.9". Scoren går fra -1 til 1.

In [25]:
tuple(blob.sentiment)[0]

-0.9

### TextBlob i funktion

Ovenstående kan nemt konverteres til funktion, som fx kunne bruges i en dataframe med `.apply`. 

Sentiment-outputtet fra TextBlob kan konverteres til en tuple, hvor den første værdi er polarity-scoren.

In [26]:
def simple_polarity(text):
    blob = TextBlob(text)
    polarity = tuple(blob.sentiment)[0] # 
    
    return(polarity)

In [27]:
simple_polarity("What an absolutely amazing weekend!")

0.7500000000000001

# ØVELSE 1: Sentiment Analysis

Foretag sentiment analysis enten af eget data eller r/Denmark data: [reddit_rdenmark-comments_01032021-08032021_long.csv](https://raw.githubusercontent.com/CALDISS-AAU/course_ddf/master/datasets/reddit_rdenmark-comments_01032021-08032021_long.csv)

(DaCy/senda for dansk, TextBlob for engelsk)

**Bemærk:** Sentiment analysis med især DaCy kan tage lang tid på større datasæt. I kan med fordel arbejde med et subset i denne øvelse, for at teste funktionerne af.

1. Anvend sentiment analysis på enkelte tekststykker
2. Anvend sentiment analysis på hele datasæt eller subet
3. Foretag opsummering af datasæt/subset, der giver indikation af, hvorvidt tonen i materialet er overvejende positiv eller negativ